In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select, WebDriverWait
import time
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os

#checking if the data file already exists
file_name = "agmarknet_data new.xlsx"
if os.path.exists(file_name):
    print(f"File '{file_name}' already exists. Exiting to avoid overwriting.")
    exit()
else:
    print(f"File '{file_name}' does not exist. Proceeding with web scraping.")   
    #IF NOT, continue with the web scraping process
    # Create a fresh Chrome session
    service = Service()
    driver = webdriver.Chrome(service=service)

    # Open a website
    driver.get("https://agmarknet.gov.in/")

    # Wait for page load
    time.sleep(3)
    # Interacting with the dropdown buttons
    select_element = driver.find_element(By.ID, "ddlCommodity") #Commodity section
    select = Select(select_element)
    select.select_by_value("103") #selecting the commodity(Ginger)
    time.sleep(2)

    # Interact with another dropdown,slecting state, district and market
    select_element = driver.find_element(By.ID, "ddlState")
    select = Select(select_element)
    select.select_by_value("KK") #selecting the state(Karnataka)
    time.sleep(3)

    select_element = driver.find_element(By.ID, "ddlDistrict") #District section
    select = Select(select_element)
    select.select_by_value("1") #selecting the district(Bagalore)
    time.sleep(3)

    select_element = driver.find_element(By.ID, "ddlMarket") #Market section
    select = Select(select_element)
    select.select_by_value("107") #selecting the market(Bangalore)
    time.sleep(3)

    #enter date in DD-MM-YYYY format

    date_input = driver.find_element(By.ID, "txtDate") # Find the date input element
    date_input.clear() # Clear any existing value
    date_input.send_keys("01-01-2010") # Type the date (Selecting first available date)
    date_input.send_keys(Keys.ENTER) # Hit Enter
    time.sleep(3)


    # Print the current URL
    print("Final URL:", driver.current_url) #Comment this line if you don't want to see the URL
    # Wait for page load
    time.sleep(3)
    # Extract table data
    data = []
    table_rows = driver.find_elements(By.XPATH, '//*[@id="cphBody_GridPriceData"]/tbody/tr') # Locate table rows using XPath

    # Iterate through rows and extract cell data
    for rw in table_rows:
        cells = rw.find_elements(By.TAG_NAME, 'td')
        row_data = []
        for cell in cells:
            text_data = cell.text.strip()
            row_data.append(text_data)
        data.append(row_data)
    driver.quit() # Close the browser
    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Export to Excel
    df.to_excel("agmarknet_data new.xlsx", index=False)

    print("✅ Data exported to agmarknet_data.xlsx")

File 'agmarknet_data new.xlsx' already exists. Exiting to avoid overwriting.


: 